Fitness Tracker App

In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import datetime
import glob
from openpyxl import load_workbook
from pathlib import Path

In [2]:
path="/Users/josephthomas/Documents/Areas of Focus/Code Repositories/git/beginners-fitness-tracking-app/Data"

In [3]:
opath="/Users/josephthomas/Documents/Areas of Focus/Code Repositories/git/beginners-fitness-tracking-app/out"

In [4]:
schedule=pd.read_csv(path+"/Schedule.csv")

In [5]:
def num_inp(size=10):
    check=0
    val=input()
    while check<1:
        if val.isnumeric():
            if int(val)<=size:
                check+=1
                return int(val)
            else:
                print("Enter a number between 0 and",size-1)
                return num_inp()
        else:
            print("Enter a valid number")
            return num_inp()
    

In [6]:
def readWOs():
    wo={}
    for file in Path(path).glob('*.csv'):
        key=str(file).split('/')[9].split('.')[0]
        value=pd.read_csv(file)
        wo[key]=value
    return wo

In [7]:
workouts=readWOs()

In [8]:
def setdate():
    a=input("Do you want to keep to schedule and complete today's exercise? (y/n)")
    if (a.lower()=='y')|(a.lower()=='yes'):
        return datetime.date.today()
    elif (a.lower()=='n')|(a.lower()=='no'):
        dat=input("Enter the date of exercise you want to complete (format: dd/mm/yyyy)")
        return datetime.datetime.strptime(dat,'%d/%m/%Y').date()
    else:
        print("Enter a valid response: ")
        return setdate()
        

In [9]:
ex_date=setdate()

Do you want to keep to schedule and complete today's exercise? (y/n)y


In [10]:
def workfunc(wo_schedule, reldate):
    weeknum=int(reldate.strftime("%U"))
    day=reldate.strftime("%A")
    wo=wo_schedule[wo_schedule['Week']==weeknum][[day,'Level']]
    numwo=len(wo)
    return wo.reset_index().drop('index',axis=1),numwo
    

In [11]:
workout,num_workout = workfunc(schedule,ex_date)

In [12]:
def wolist(WO,num_WO):
    temp=[]
    day=ex_date.strftime("%A")
    for i in range(num_WO):
        tempdf=workouts[WO[day][i]]
        temp.append(tempdf[tempdf.level==WO['Level'][i]])
    temp=pd.concat(temp).reset_index().drop('index',axis=1)
    return temp
    

In [13]:
days_exercise=wolist(workout,num_workout)

In [14]:
print(days_exercise)

                             exercise             type  level
0                       DB Drop Squat            squat      1
1                RA DB Overhead Press    vertical push      1
2                LA DB Overhead Press    vertical push      1
3                 Chest Supported Row  horizontal pull      1
4                      BW Split Squat     static lunge      1
5  Thumbs Up Dumbell Bench Press +2kg          Triceps      2


In [15]:
input()

''

In [16]:
def finaldb(exercise):
    a=[]
    i=0
    while i<exercise.shape[0]:
        print("Exercise: ", exercise.loc[i,"exercise"])
        print("Type: ", exercise.loc[i,"type"])
        print("Enter number of sets: ")
        max_sets=num_inp(10)
        reps=" "
        weight=[]
        if max_sets!=0:
            for j in range(max_sets):
                print("Enter number of reps in set",j+1,": ")
                rep=str(num_inp(50))
                reps=reps+rep
                if j!=(max_sets-1):
                    reps+="x"
                print("Enter weight used in set", j+1,": ")
                weight.append(num_inp(400))
        else:
            reps="0x0x0"
            weight=[0,0,0]
        exercise.loc[i,"sets"]=max_sets
        exercise.loc[i,"reps"]=reps
        exercise.loc[i,"max_weight"]=np.array(weight).max()
        exercise.loc[i,"avg_weight"]=np.array(weight).mean()
        i+=1
    return exercise
    

In [17]:
days_exercise=finaldb(days_exercise)

Exercise:  DB Drop Squat
Type:  squat
Enter number of sets: 
3
Enter number of reps in set 1 : 
13
Enter weight used in set 1 : 
20
Enter number of reps in set 2 : 
12
Enter weight used in set 2 : 
22
Enter number of reps in set 3 : 
13
Enter weight used in set 3 : 
22
Exercise:  RA DB Overhead Press
Type:  vertical push
Enter number of sets: 
3
Enter number of reps in set 1 : 
12
Enter weight used in set 1 : 
10
Enter number of reps in set 2 : 
12
Enter weight used in set 2 : 
10
Enter number of reps in set 3 : 
10
Enter weight used in set 3 : 
10
Exercise:  LA DB Overhead Press
Type:  vertical push
Enter number of sets: 
3
Enter number of reps in set 1 : 
15
Enter weight used in set 1 : 
8
Enter number of reps in set 2 : 
12
Enter weight used in set 2 : 
7
Enter number of reps in set 3 : 
12
Enter weight used in set 3 : 
6
Exercise:  Chest Supported Row
Type:  horizontal pull
Enter number of sets: 
3
Enter number of reps in set 1 : 
15
Enter weight used in set 1 : 
6
Enter number of 

In [18]:
days_exercise['date'] = ex_date
days_exercise=days_exercise[['date','exercise','type','level','sets','avg_weight','max_weight','reps']]

In [19]:
def adddata(df, adate):
    filename=opath+'/{}.xlsx'.format(adate.strftime('%B'))
    if Path(filename).is_file():
        writer=pd.ExcelWriter(filename, engine='openpyxl', mode='a')
        writer.book = load_workbook(filename)
        startrow = writer.book['Sheet1'].max_row
        writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
        df.to_excel(writer, 'Sheet1', startrow=startrow,header=None,index=False)
        writer.save()
    else:
        df.to_excel(filename, index=False)

In [20]:
adddata(days_exercise,ex_date)

Part B: Analysis And Visualization

import matplotlib.pyplot as plt

df=pd.read_excel(opath+'/{}.xlsx'.format(datetime.date.today().strftime('%B')))

plt.plot(df.groupby(['date','type']).mean().reset_index()['date'],df.groupby(['date','type']).mean().reset_index()['max_weight'])

df.groupby(['date','type']).mean().reset_index()['date']